In [1]:
import carla
import argparse
import time
import signal
import sys
import traceback
import os
import subprocess
import re
import socket
import atexit
import threading
from datetime import datetime
from xml.etree import ElementTree as ET
import py_trees


# os.environ['CUDA_HOME'] = '/usr/local/cuda-12.2'
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['REPETITION'] = '1'
# os.environ['DATAGEN'] = '1'


In [2]:
import subprocess, re

def kill_carla_processes_on_gpu(gpu_index: int = 0):
    """Kill CARLA processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\+G\s+.*CarlaUE4'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No CarlaUE4 processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing CarlaUE4 process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing CARLA processes: {e}")

def kill_python_processes_on_gpu(gpu_index: int = 0):
    """Kill Python processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\s+.*bin/python'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No python processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing python process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing Python processes: {e}")

def cleanup_gpu_processes():
    """Clean up GPU processes"""
    for gpu_id in [0, 1]:
        kill_carla_processes_on_gpu(gpu_id)
        kill_python_processes_on_gpu(gpu_id)

cleanup_gpu_processes()

Killing CarlaUE4 process with PID 392211 on GPU 0
No python processes found on GPU 0
No CarlaUE4 processes found on GPU 1
No python processes found on GPU 1


In [3]:
# Setup paths
leaderboard_path = '/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard'
if leaderboard_path not in sys.path:
    sys.path.insert(0, leaderboard_path)

# Check environment variables
print("CARLA_ROOT: ", os.environ.get('CARLA_ROOT'))
print("WORK_DIR: ", os.environ.get('WORK_DIR'))
print("SCENARIO_RUNNER_ROOT: ", os.environ.get('SCENARIO_RUNNER_ROOT'))
print("LD_LIBRARY_PATH: ", os.environ.get('LD_LIBRARY_PATH'))
print("LEADERBOARD_ROOT: ", os.environ.get('LEADERBOARD_ROOT'))
print("CUDA_HOME: ", os.environ.get('CUDA_HOME'))
print("PYTHONPATH: ", os.environ.get('PYTHONPATH'))

CARLA_ROOT:  /mnt3/Documents/AD_Framework/carla0915
WORK_DIR:  /mnt3/Documents/AD_Framework/Bench2Drive
SCENARIO_RUNNER_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner
LD_LIBRARY_PATH:  /usr/local/cuda-12.1/lib64:
LEADERBOARD_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard
CUDA_HOME:  /usr/local/cuda-12.1
PYTHONPATH:  /mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla


In [4]:
port = 20019

model_name = 'my_model_mlp'
statistics_manager_path = f'/mnt3/Documents/AD_Framework/results/{model_name}'

# makedir if do not exist
os.makedirs(statistics_manager_path, exist_ok=True)

args = argparse.Namespace()
args.WORK_DIR = os.environ.get('WORK_DIR')
args.checkpoint = f'{statistics_manager_path}/stat_manager_results.json'
args.debug_checkpoint = f'{statistics_manager_path}/live_results.txt'
args.host = '127.0.0.3'
args.port = port
args.timeout = 6000
args.frame_rate = 20.0 # fixed
args.gpu_rank = 0
args.traffic_manager_port = port + 3
# '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent.py'
# args.TEAM_AGENT = f"{args.WORK_DIR}/leaderboard/team_code/data_agent2.py" # auto_pilot.py
args.TEAM_AGENT = f'{leaderboard_path}/autoagents/dummy_agent4.py'
## auto_pilot.py
args.agent = args.TEAM_AGENT
print("args.agent", args.agent)
args.agent_config = ''
args.debug = 1
args.routes = f"/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml"
args.repetitions = 1
args.track = 'SENSORS' # Changed from 'MAP' to 'SENSORS' to allow camera sensors
args.record = ''
args.routes_subset = 0
args.resume = 0
args.traffic_manager_seed = port - 3333 + 1

# Enable camera sensors by setting required environment variables
os.environ['SAVE_PATH'] = statistics_manager_path
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['TMP_VISU'] = '0'  # Enable camera sensors for visualization

print(f"SAVE_PATH set to: {statistics_manager_path}")
print(f"TMP_VISU set to: {os.environ.get('TMP_VISU')}")
print(f"Track set to: {args.track}")

args.agent /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent4.py
SAVE_PATH set to: /mnt3/Documents/AD_Framework/results/my_model_mlp
TMP_VISU set to: 0
Track set to: SENSORS


In [5]:
from srunner.scenariomanager.carla_data_provider import *
from srunner.scenariomanager.timer import GameTime
from srunner.scenariomanager.watchdog import Watchdog

from leaderboard.scenarios.scenario_manager import ScenarioManager
from leaderboard.scenarios.route_scenario import RouteScenario
from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
from leaderboard.autoagents.agent_wrapper import AgentError, validate_sensor_configuration, TickRuntimeError
from leaderboard.utils.statistics_manager import StatisticsManager, FAILURE_MESSAGES
from leaderboard.utils.route_indexer import RouteIndexer

# print("args: ", args)
statistics_manager = StatisticsManager(args.checkpoint , args.debug_checkpoint)
# statistics_manager.write_statistics() ## run one time

# leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

LeaderboardEvaluator_path = '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/leaderboard_evaluator.py'
if LeaderboardEvaluator_path not in sys.path:
    sys.path.insert(0, LeaderboardEvaluator_path)

from leaderboard.leaderboard_evaluator import LeaderboardEvaluator


In [6]:
import socket
import subprocess
import atexit


def find_free_port(starting_port):
    """Find a free port starting from the given port"""
    port = starting_port
    while True:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(("localhost", port))
                return port
        except OSError:
            port += 1

def _setup_simulation(self, args):
    """Setup simulation with CARLA server"""
    os.system("pkill -f CarlaUE4")
    time.sleep(5)

    self.carla_path = os.environ["CARLA_ROOT"]
    args.port = find_free_port(args.port)
    cmd1 = f"{os.path.join(self.carla_path, 'CarlaUE4.sh')} -RenderOffScreen -nosound -quality-level=Epic -carla-rpc-port={args.port} -carla-streaming-port=0 -graphicsadapter=0"
    self.server = subprocess.Popen(cmd1, shell=True, preexec_fn=os.setsid)
    atexit.register(os.killpg, self.server.pid, signal.SIGKILL)

    # Wait for server to be ready
    server_ready = False
    for _ in range(30):
        try:
            with socket.create_connection(('localhost', args.port), timeout=1):
                server_ready = True
                break
        except (ConnectionRefusedError, socket.timeout):
            time.sleep(5)
    if not server_ready:
        raise RuntimeError("Carla server failed to start")
        
    attempts = 0
    num_max_restarts = 20
    print("Loading world!!!")
    while attempts < num_max_restarts:
        try:
            client = carla.Client(args.host, args.port)
            if args.timeout:
                client_timeout = args.timeout
            client.set_timeout(client_timeout)

            settings = carla.WorldSettings(
                synchronous_mode = True,
                fixed_delta_seconds = 1.0 / self.frame_rate,
                deterministic_ragdolls = True,
                spectator_as_ego = False
            )
            client.get_world().apply_settings(settings)
            print(f"load_world success , attempts={attempts}", flush=True)
            break
        except Exception as e:
            print(f"load_world failed , attempts={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)
    
    attempts = 0
    num_max_restarts = 40
    while attempts < num_max_restarts:
        try:
            args.traffic_manager_port = find_free_port(args.traffic_manager_port)
            traffic_manager = client.get_trafficmanager(args.traffic_manager_port)
            traffic_manager.set_synchronous_mode(True)
            traffic_manager.set_hybrid_physics_mode(True)
            print(f"traffic_manager init success, try_time={attempts}", flush=True)
            break
        except Exception as e:
            print(f"traffic_manager init fail, try_time={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)

    return client, client_timeout, traffic_manager


def get_weather_id(weather_conditions):
    from xml.etree import ElementTree as ET
    WORK_DIR = os.environ.get("WORK_DIR", "")
    if WORK_DIR != "":
        WORK_DIR += "/"
    tree = ET.parse(WORK_DIR + 'leaderboard/data/weather.xml')
    root = tree.getroot()
    def conditions_match(weather, conditions):
        for (key, value) in weather:
            if key == 'route_percentage' : continue
            if str(getattr(conditions, key))!= value:
                return False
        return True
    for case in root.findall('case'):
        weather = case[0].items()
        if conditions_match(weather, weather_conditions):
            return case.items()[0][1]
    return None

sensors_to_icons = {
    'sensor.camera.rgb':        'carla_camera',
    'sensor.lidar.ray_cast':    'carla_lidar',
    'sensor.other.radar':       'carla_radar',
    'sensor.other.gnss':        'carla_gnss',
    'sensor.other.imu':         'carla_imu',
    'sensor.opendrive_map':     'carla_opendrive_map',
    'sensor.speedometer':       'carla_speedometer'
}

#### the following are the functions that equal to _load_and_run_scenario in leaderboard_evaluator.py

def my_load_scenario(leaderboard_evaluator_self, args, config):
    """Load and run the scenario given by config"""
    crash_message = ""
    entry_status = "Started"
    save_name = ""

    print("\n\033[1m========= Preparing {} (repetition {}) =========\033[0m".format(config.name, config.repetition_index), flush=True)

    # Prepare the statistics of the route
    route_name = f"{config.name}_rep{config.repetition_index}"
    scenario_name = config.scenario_configs[0].name
    town_name = str(config.town)
    weather_id = get_weather_id(config.weather[0][1])
    currentDateAndTime = datetime.now()
    currentTime = currentDateAndTime.strftime("%m_%d_%H_%M_%S")
    save_name = f"{route_name}_{town_name}_{scenario_name}_{weather_id}_{currentTime}"
    leaderboard_evaluator_self.statistics_manager.create_route_data(route_name, scenario_name, weather_id, save_name, town_name, config.index)

    print("\033[1m> Loading the world\033[0m", flush=True)

    # Load the world and the scenario
    try:
        leaderboard_evaluator_self._load_and_wait_for_world(args, config.town)
        from leaderboard.scenarios.route_scenario import RouteScenario
        leaderboard_evaluator_self.route_scenario = RouteScenario(world=leaderboard_evaluator_self.world, config=config, debug_mode=args.debug)
        leaderboard_evaluator_self.statistics_manager.set_scenario(leaderboard_evaluator_self.route_scenario)

    except Exception:
        # The scenario is wrong -> set the ejecution to crashed and stop
        print("\n\033[91mThe scenario could not be loaded:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, save_name, entry_status, crash_message

    return False, save_name, entry_status, crash_message

def my_load_agent(leaderboard_evaluator_self, args, config, save_name, entry_status, crash_message):
    """Set up the user's agent and configure sensors"""
    print("\033[1m> Setting up the agent\033[0m", flush=True)

    # Set up the user's agent, and the timer to avoid freezing the simulation
    try:
        from srunner.scenariomanager.watchdog import Watchdog
        from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
        from leaderboard.autoagents.agent_wrapper import validate_sensor_configuration
        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        
        leaderboard_evaluator_self._agent_watchdog = Watchdog(args.timeout)
        leaderboard_evaluator_self._agent_watchdog.start()
        agent_class_name = getattr(leaderboard_evaluator_self.module_agent, 'get_entry_point')()
        agent_class_obj = getattr(leaderboard_evaluator_self.module_agent, agent_class_name)

        # Start the ROS1 bridge server only for ROS1 based agents.
        if getattr(agent_class_obj, 'get_ros_version')() == 1 and leaderboard_evaluator_self._ros1_server is None:
            from leaderboard.autoagents.ros1_agent import ROS1Server
            leaderboard_evaluator_self._ros1_server = ROS1Server()
            leaderboard_evaluator_self._ros1_server.start()

        leaderboard_evaluator_self.agent_instance = agent_class_obj(args.host, args.port, args.debug)
        leaderboard_evaluator_self.agent_instance.set_global_plan(leaderboard_evaluator_self.route_scenario.gps_route, leaderboard_evaluator_self.route_scenario.route)
        args.agent_config = args.agent_config + '+' + save_name
        leaderboard_evaluator_self.agent_instance.setup(args.agent_config)

        # Check and store the sensors
        if not leaderboard_evaluator_self.sensors:
            leaderboard_evaluator_self.sensors = leaderboard_evaluator_self.agent_instance.sensors()
            track = leaderboard_evaluator_self.agent_instance.track

            validate_sensor_configuration(leaderboard_evaluator_self.sensors, track, args.track)

            leaderboard_evaluator_self.sensor_icons = [sensors_to_icons[sensor['type']] for sensor in leaderboard_evaluator_self.sensors]
            leaderboard_evaluator_self.statistics_manager.save_sensors(leaderboard_evaluator_self.sensor_icons)
            leaderboard_evaluator_self.statistics_manager.write_statistics()

            leaderboard_evaluator_self.sensors_initialized = True

        leaderboard_evaluator_self._agent_watchdog.stop()
        leaderboard_evaluator_self._agent_watchdog = None

    except SensorConfigurationInvalid as e:
        # The sensors are invalid -> set the ejecution to rejected and stop
        print("\n\033[91mThe sensor's configuration used is invalid:", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Sensors"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    except Exception as e:
        # The agent setup has failed -> start the next route
        print("\n\033[91mCould not set up the required agent:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Agent_init"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    return False, entry_status, crash_message

def my_load_route_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Load the route scenario and prepare for execution"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    print("\033[1m> Running the route\033[0m", flush=True)

    # Run the scenario
    try:
        # Load scenario and run it
        if args.record:
            leaderboard_evaluator_self.client.start_recorder("{}/{}_rep{}.log".format(args.record, config.name, config.repetition_index))
        leaderboard_evaluator_self.manager.load_scenario(leaderboard_evaluator_self.route_scenario, leaderboard_evaluator_self.agent_instance, config.index, config.repetition_index)
    except Exception:
        print("\n\033[91mFailed to load the scenario, the statistics might be empty:", flush=True)
        print("\n\033[91mLoading the route, the agent has crashed:", flush=True)
        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

    return False, entry_status, crash_message

def my_run_scenario_setup(leaderboard_evaluator_self):
    """Trigger the start of the scenario and wait for it to finish/fail"""
    from srunner.scenariomanager.timer import GameTime
    from srunner.scenariomanager.watchdog import Watchdog
    
    leaderboard_evaluator_self.manager.tick_count = 0
    leaderboard_evaluator_self.manager.start_system_time = time.time()
    leaderboard_evaluator_self.manager.start_game_time = GameTime.get_time()

    # Detects if the simulation is down
    leaderboard_evaluator_self.manager._watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._watchdog.start()

    # Stop the agent from freezing the simulation
    leaderboard_evaluator_self.manager._agent_watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._agent_watchdog.start()

    leaderboard_evaluator_self.manager._running = True

    # Thread for build_scenarios
    leaderboard_evaluator_self.manager._scenario_thread = threading.Thread(target=leaderboard_evaluator_self.manager.build_scenarios_loop, args=(leaderboard_evaluator_self.manager._debug_mode > 0, ))
    leaderboard_evaluator_self.manager._scenario_thread.start()

def my_run_scenario_step(leaderboard_evaluator_self, entry_status, crash_message, n_steps=1):
    """Run n_steps of the scenario simulation"""
    from leaderboard.autoagents.agent_wrapper import AgentError, TickRuntimeError
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    try:
        if leaderboard_evaluator_self.manager._running:
            print("In my_run_scenario_step (Check if it is stuck or not)")
            for _ in range(n_steps):
                leaderboard_evaluator_self.manager._tick_scenario()

    except AgentError:
        # The agent has failed -> stop the route
        print("\n\033[91mStopping the route, the agent has crashed:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m")

        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

        return True, entry_status, crash_message

    except KeyboardInterrupt:
        return True, entry_status, crash_message
    
    except TickRuntimeError:
        print("TickRuntimeError")
        entry_status, crash_message = "Started", "TickRuntime"
        return True, entry_status, crash_message
    
    except Exception:
        print("\n\033[91mError during the simulation:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        return True, entry_status, crash_message
    
    return False, entry_status, crash_message

def my_stop_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Stop the scenario and register statistics"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    # Stop the scenario
    try:
        print("\033[1m> Stopping the route\033[0m", flush=True)
        leaderboard_evaluator_self.manager.stop_scenario()
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)

        if args.record:
            leaderboard_evaluator_self.client.stop_recorder()

        leaderboard_evaluator_self._cleanup()

    except Exception:
        print("\n\033[91mFailed to stop the scenario, the statistics might be empty:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        _, crash_message = FAILURE_MESSAGES["Simulation"]

    return False, entry_status, crash_message

# replace _setup_simulation method of LeaderboardEvaluator with this new one 
LeaderboardEvaluator._setup_simulation = _setup_simulation


In [7]:
### Main code object

leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

4.26.2-0+++UE4+Release-4.26 522 0
Disabling core dumps.
Loading world!!!
load_world success , attempts=0
traffic_manager init success, try_time=0


In [8]:
print("args.routes", args.routes)
print("args.repetitions", args.repetitions)
print("args.routes_subset", args.routes_subset)

args.routes /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml
args.repetitions 1
args.routes_subset 0


In [9]:
route_indexer = RouteIndexer(args.routes, args.repetitions, args.routes_subset)
if args.resume:
    resume = route_indexer.validate_and_resume(args.checkpoint)
else:
    resume = False

if resume:
    leaderboard_evaluator.statistics_manager.add_file_records(args.checkpoint)
else:
    leaderboard_evaluator.statistics_manager.clear_records()
leaderboard_evaluator.statistics_manager.save_progress(route_indexer.index, route_indexer.total)
leaderboard_evaluator.statistics_manager.write_statistics()

In [10]:
crashed_flag = False
t1 = time.time()
route_count = 0

In [11]:
### in the loop of running

want_list = ['1790'] #['25845']

for i in range(250):
    config = route_indexer.get_next_config()
    if config is None:
        print("No more routes to process")
        

    route_id = config.name.split("_")[1]
    print("route_id: ", route_id)
    if route_id in want_list and len(want_list) > 0:
        print("Got the route_id: ", route_id)
        break
    elif len(want_list) == 0:
        route_id = route_id
        print("Got the route_id: ", route_id)
        break
    
print(f"\n=== Processing Route {route_count + 1}/{route_indexer.total} ===")
print(f"Route: {config.name}, Town: {config.town}", "Route_index: ", route_indexer.index)

### want route_id = 2509

route_id:  1711
route_id:  1773
route_id:  1790
Got the route_id:  1790

=== Processing Route 1/220 ===
Route: RouteScenario_1790, Town: Town12 Route_index:  3


In [12]:
# Step 1: Load scenario

crash_flag, save_name, entry_status, crash_message = my_load_scenario(leaderboard_evaluator, args, config)
print("crash_flag: ", crash_flag)
print("save_name: ", save_name)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

save_name = save_name + '_' + model_name


========= Preparing RouteScenario_1790 (repetition 0) =========
> Loading the world
crash_flag:  False
save_name:  RouteScenario_1790_rep0_Town12_HazardAtSideLane_1_8_09_03_14_55_31
entry_status:  Started
crash_message:  


In [13]:
# Step 2: Load agent
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_agent(leaderboard_evaluator, args, config, save_name, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Setting up the agent
DummyAgent2 initialized - will save 10 images to /mnt3/Documents/AD_Framework/town_test_images
crash_flag:  False
entry_status:  Started
crash_message:  


In [14]:
# Step 3: Load route scenario
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_route_scenario(leaderboard_evaluator, args, config, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Running the route
crash_flag:  False
entry_status:  Started
crash_message:  


In [15]:
# Step 4: Setup scenario
if not crash_flag:
    my_run_scenario_setup(leaderboard_evaluator)

# Add image saving capability to the agent
import numpy as np
from datetime import datetime

# Monkey-patch the agent to capture sensor data
original_run_step = leaderboard_evaluator.agent_instance.run_step

def enhanced_run_step(input_data, timestamp):
    """Enhanced run_step that captures sensor data for image saving"""
    # Store the input data for later access
    leaderboard_evaluator.agent_instance.last_input_data = input_data
    leaderboard_evaluator.agent_instance.step = getattr(leaderboard_evaluator.agent_instance, 'step', 0) + 1
    
    # Call original run_step
    return original_run_step(input_data, timestamp)

# Replace the run_step method
leaderboard_evaluator.agent_instance.run_step = enhanced_run_step
leaderboard_evaluator.agent_instance.step = 0

print("Agent enhanced with image capture capability")

Agent enhanced with image capture capability


In [16]:
# Step 5: Run scenario loop step by step with automatic image saving
import cv2
import numpy as np
from pathlib import Path
from datetime import datetime

leaderboard_evaluator.manager._running = True

# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

def save_debug_images(agent_instance, save_dir, step_count):
    """Save all sensor images for debugging with velocity overlay"""
    if not hasattr(agent_instance, 'last_input_data'):
        return

    input_data = agent_instance.last_input_data
    # leaderboard_evaluator.agent_instance.last_input_data['speed'][1]['speed']

    # Create step-specific subdirectory
    step_dir = save_dir / f"step_{step_count:04d}"
    step_dir.mkdir(exist_ok=True)

    # Extract velocity data from sensors
    ego_speed = None
    ego_velocity = None

    # Try to get speed from SPEED sensor
    if 'speed' in input_data:
        speed_data = input_data['speed']
        if isinstance(speed_data, tuple) and len(speed_data) >= 2:
            if isinstance(speed_data[1], dict) and 'speed' in speed_data[1]:
                ego_speed = speed_data[1]['speed']  # m/s
            elif isinstance(speed_data[1], (int, float)):
                ego_speed = float(speed_data[1])

    # Try to get velocity from IMU sensor
    if 'IMU' in input_data:
        imu_data = input_data['IMU']
        if isinstance(imu_data, tuple) and len(imu_data) >= 2:
            if isinstance(imu_data[1], dict):
                if 'accelerometer' in imu_data[1]:
                    # IMU provides acceleration, not velocity directly
                    pass

    # Try to get GPS data for position tracking
    gps_data = None
    if 'GPS' in input_data:
        gps = input_data['GPS']
        if isinstance(gps, tuple) and len(gps) >= 2:
            if isinstance(gps[1], dict):
                gps_data = gps[1]

    # Save RGB cameras with velocity overlay
    for camera_id in ['Center', 'Left', 'Right']:
        if camera_id in input_data:
            rgb_image = input_data[camera_id][1][:, :, :3]  # Remove alpha channel if present
            frame_num = input_data[camera_id][0]

            # Convert from RGB to BGR for OpenCV
            bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

            # Add velocity overlay on Center camera
            if camera_id == 'Center':
                # Create overlay text
                overlay_text = []
                overlay_text.append(f"Step: {step_count:04d}")

                # Add ego vehicle speed
                if ego_speed is not None:
                    # Convert m/s to km/h for better readability
                    speed_kmh = ego_speed * 3.6
                    overlay_text.append(f"Ego Speed: {speed_kmh:.1f} km/h ({ego_speed:.1f} m/s)")
                else:
                    overlay_text.append("Ego Speed: N/A")

                # Add GPS info if available
                if gps_data:
                    lat = gps_data.get('lat', 0)
                    lon = gps_data.get('lon', 0)
                    alt = gps_data.get('alt', 0)
                    overlay_text.append(f"GPS: ({lat:.6f}, {lon:.6f}, {alt:.1f}m)")

                # Draw overlay on image
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.7
                thickness = 2
                y_offset = 30

                for i, text in enumerate(overlay_text):
                    # Add black background for text visibility
                    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                    cv2.rectangle(bgr_image,
                                (10, y_offset + i*35 - text_height - 5),
                                (10 + text_width + 10, y_offset + i*35 +5),
                                (0, 0, 0), -1)

                    # Draw white text
                    cv2.putText(bgr_image, text,
                            (15, y_offset + i*35),
                            font, font_scale, (255, 255, 255), thickness)

            # Save image
            image_path = step_dir / f"{camera_id}_frame_{frame_num:06d}.jpg"
            cv2.imwrite(str(image_path), bgr_image)
            # print(f"  Saved: {camera_id} -> {image_path.name}")

    # Save sensor data info to text file including velocity
    info_path = step_dir / "sensor_info.txt"
    with open(info_path, 'w') as f:
        f.write(f"Step: {step_count}\n")
        f.write(f"Timestamp: {datetime.now()}\n")
        if ego_speed is not None:
            f.write(f"Ego Speed: {ego_speed:.2f} m/s ({ego_speed*3.6:.1f} km/h)\n")
        if gps_data:
            f.write(f"GPS: lat={gps_data.get('lat', 0):.6f}, lon={gps_data.get('lon', 0):.6f}, alt={gps_data.get('alt', 0):.1f}\n")
        f.write("\nSensor Data:\n")
        for key, val in input_data.items():
            if hasattr(val[1], 'shape'):
                f.write(f"  {key}: frame={val[0]:06d}, shape={val[1].shape}\n")
            else:
                f.write(f"  {key}: frame={val[0]:06d}\n")

    return step_dir


# Run simulation steps with image saving
run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

# Display summary
if save_dir.exists():
    step_dirs = sorted(save_dir.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} steps of data")
    for step_dir in step_dirs[:3]:  # Show first 3 steps
        images = list(step_dir.glob("*.jpg"))
        print(f"  {step_dir.name}: {len(images)} images")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_145625

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:25.273 -- System time = 0.000 -- Game time = 0.050 -- Ratio = 0.000x
Testing Test_0 - Route_0
Frame 1/10 - Test_0
  Saving Center camera: Test_0_Route_0_Center_frame_001.png
  Saving Left camera: Test_0_Route_0_Left_frame_001.png
  Saving Right camera: Test_0_Route_0_Right_frame_001.png

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:26.096 -- System time = 0.823 -- Game time = 0.100 -- Ratio = 0.122x
Frame 2/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:26.829 -- System time = 1.555 -- Game time = 0.150 -- Ratio = 0.096x
Frame 3/10 - Test_0

Step 2:


In [17]:
#### save snapshot

import json
import csv     

world_obj = leaderboard_evaluator.world

def record_vehicles(world_obj, save_path=None, save_json=False):
    actors = world_obj.get_actors()
    vehicles = actors.filter("vehicle.*")

    records = []
    for v in vehicles:
        tf = v.get_transform()
        loc, rot = tf.location, tf.rotation
        vel = v.get_velocity()
        ang = v.get_angular_velocity()
        acc = v.get_acceleration()
        ctrl = v.get_control()

        record = {
            "id": v.id,
            "type_id": v.type_id,
            "x": loc.x,
            "y": loc.y,
            "z": loc.z,
            "pitch": rot.pitch,
            "yaw": rot.yaw,
            "roll": rot.roll,
            "vx": vel.x, "vy": vel.y, "vz": vel.z,
            "wx": ang.x, "wy": ang.y, "wz": ang.z,
            "ax": acc.x, "ay": acc.y, "az": acc.z,
            "throttle": ctrl.throttle,
            "steer": ctrl.steer,
            "brake": ctrl.brake,
            "hand_brake": ctrl.hand_brake,
            "reverse": ctrl.reverse,
            "gear": ctrl.gear,
        }
        records.append(record)

    if save_path:
        if save_json:
            import json
            with open(save_path, "w") as f:
                json.dump(records, f, indent=4)
            print(f"Saved {len(records)} vehicle records to {save_path} (JSON)")
        else:
            import csv
            with open(save_path, mode="w", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=records[0].keys())
                writer.writeheader()
                writer.writerows(records)
            print(f"Saved {len(records)} vehicle records to {save_path} (CSV)")

    return records

def watchdog_save_state(watchdog_object):
    record_dict = {
        "timeout": watchdog_object._timeout,
        "interval": watchdog_object._interval,
        "failed": watchdog_object._failed,
        "stopped": watchdog_object._watchdog_stopped,
    }
    return record_dict

### to save state 
vehicles_state = record_vehicles(world_obj)
print("vehicles_state", vehicles_state)
print("--------------------------------")

watchdog_state = watchdog_save_state(leaderboard_evaluator.manager._watchdog)
print("watchdog_state", watchdog_state)
print("--------------------------------")
    

vehicles_state [{'id': 3771, 'type_id': 'vehicle.lincoln.mkz_2020', 'x': 529.6852416992188, 'y': 3920.765625, 'z': 371.2006530761719, 'pitch': 0.43278124928474426, 'yaw': 0.11760231107473373, 'roll': 0.016277821734547615, 'vx': 6.651047706604004, 'vy': 0.005099917761981487, 'vz': 0.025384750217199326, 'wx': -0.04707903042435646, 'wy': -4.131840229034424, 'wz': -0.38689517974853516, 'ax': 12.258825302124023, 'ay': -0.0012694299221038818, 'az': 0.03517858684062958, 'throttle': 0.8500000238418579, 'steer': -0.002491297898814082, 'brake': 0.0, 'hand_brake': False, 'reverse': False, 'gear': 2}, {'id': 3770, 'type_id': 'vehicle.mini.cooper_s_2021', 'x': 529.888671875, 'y': 3917.51708984375, 'z': 371.2105712890625, 'pitch': 0.18733158707618713, 'yaw': 0.0877789556980133, 'roll': -9.155272709904239e-05, 'vx': 6.486534118652344, 'vy': 0.002112228889018297, 'vz': 0.02569270133972168, 'wx': -0.00952806044369936, 'wy': 0.2599191665649414, 'wz': -0.8037283420562744, 'ax': 1.7831707000732422, 'ay': 

In [18]:

#### second 50s

# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_145639

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:39.872 -- System time = 14.599 -- Game time = 2.550 -- Ratio = 0.175x
Frame 51/10 - Test_0

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:40.074 -- System time = 14.801 -- Game time = 2.600 -- Ratio = 0.176x
Frame 52/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:40.270 -- System time = 14.996 -- Game time = 2.650 -- Ratio = 0.177x
Frame 53/10 - Test_0

Step 2:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:40.467 -- System time = 15.194 -- Game time = 2.700 -- Ratio = 0.178x
Frame 54/10 - Test_0

Ste

In [19]:
import carla, threading, time

def stop_manager(manager):
    manager._running = False

    t = getattr(manager, "_scenario_thread", None)
    if t and hasattr(t, "is_alive") and t.is_alive() and threading.current_thread() is not t:
        t.join(timeout=2.0)
    manager._scenario_thread = None

    # if a tick thread ever existed, stop it too (safe even if missing)
    tt = getattr(manager, "_tick_thread", None)
    if tt and hasattr(tt, "is_alive") and tt.is_alive() and threading.current_thread() is not tt:
        tt.join(timeout=2.0)
    manager._tick_thread = None

def reset_tree(tree):
    for node in tree.iterate():  # breadth-first
        try:
            node.terminate(py_trees.common.Status.INVALID)
        except Exception:
            pass
        try:
            node.initialise()
        except Exception:
            pass
    tree.status = py_trees.common.Status.RUNNING

def reinit_inroute_and_blocked(tree):
    """Re-initialise only InRouteTest and ActorBlockedTest (safe, no spawns)."""
    target = {"InRouteTest", "ActorBlockedTest"}
    for n in tree.iterate():
        if n.__class__.__name__ in target:
            try:
                n.terminate(py_trees.common.Status.INVALID)  # clear latched status
                n.initialise()                               # re-arm the check
                # print(f"reinit: {n.__class__.__name__} -> ok")  # optional
            except Exception:
                pass

def _existing_vehicle_ids(world):
    return {v.id for v in world.get_actors().filter("vehicle.*")}

def _vec3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))

def start_manager_builder_only(manager):
    """Start ONLY the builder loop; NO auto-ticking."""
    manager._running = True
    t = getattr(manager, "_scenario_thread", None)
    if t is None or not (hasattr(t, "is_alive") and t.is_alive()):
        manager._scenario_thread = threading.Thread(
            target=manager.build_scenarios_loop,
            args=(manager._debug_mode > 0,),
            daemon=True
        )
        manager._scenario_thread.start()
    # <-- DO NOT call _tick_scenario() here; you will step manually.

def _to_transform(entry):
    if isinstance(entry, tuple) and len(entry) >= 1:
        entry = entry[0]

    if isinstance(entry, carla.Transform):
        return entry

    if hasattr(entry, "transform"):
        return entry.transform

    if isinstance(entry, dict):
        x = float(entry.get("x", 0.0))
        y = float(entry.get("y", 0.0))
        z = float(entry.get("z", 0.0))
        yaw = float(entry.get("yaw", 0.0))
        return carla.Transform(
            carla.Location(x, y, z),
            carla.Rotation(yaw=yaw)
        )
    raise TypeError(f"Don't know how to convert plan entry {type(entry)} to carla.Transform")

def _v3(x=0, y=0, z=0):
    return carla.Vector3D(float(x), float(y), float(z))


def pause_restore_resume(client, world, manager, vehicles_state, mode="snapshot_strict",  
    apply_controls=True,         
    keep_sync=True               
):
    # 1) Pause any manager activity
    stop_manager(manager)

    # 2) Ensure sync (paused) and remember previous settings
    prev = world.get_settings()
    changed = False
    if not prev.synchronous_mode:
        new = carla.WorldSettings()
        new.no_rendering_mode   = prev.no_rendering_mode
        new.synchronous_mode    = True
        new.fixed_delta_seconds = prev.fixed_delta_seconds or 0.05
        world.apply_settings(new)
        changed = True

    # 3) Build atomic batch
    present = _existing_vehicle_ids(world)
    
    # --- MAIN RESTORE BATCH ---
    batch = []

    for rec in vehicles_state:
        vid = int(rec["id"])
        if vid not in present:
            continue

        tf = carla.Transform(
            carla.Location(float(rec["x"]), float(rec["y"]), float(rec["z"])),
            carla.Rotation(
                yaw=float(rec.get("yaw", 0.0)),
                pitch=float(rec.get("pitch", 0.0)),
                roll=float(rec.get("roll",  0.0)),
            )
        )

        # Disable physics, set transform, re-enable physics
        batch += [
            carla.command.SetSimulatePhysics(vid, False),
            carla.command.ApplyTransform(vid, tf),
            carla.command.SetSimulatePhysics(vid, True),
        ]

    # Apply transforms first
    client.apply_batch_sync(batch, True)
    world.tick()  # Let physics settle

    # Apply velocities using set_target_velocity directly (more reliable)
    if mode == "snapshot_strict":
        for rec in vehicles_state:
            vid = int(rec["id"])
            if vid not in present:
                continue

            actor = world.get_actor(vid)
            if actor:
                # Get saved velocities
                vx = abs(rec.get("vx", 0.0))
                vy = abs(rec.get("vy", 0.0))
                vz = abs(rec.get("vz", 0.0))
                wx = rec.get("wx", 0.0)
                wy = rec.get("wy", 0.0)
                wz = rec.get("wz", 0.0)
                # print("vx, vy, vz", vx, vy, vz)

                # Debug print
                if vid == 3695:
                    v_cur = actor.get_velocity()
                    print(f"Ego before impulse: cur=({v_cur.x:.2f}, {v_cur.y:.2f}, {v_cur.z:.2f}), target=({vx:.2f}, {vy:.2f}, {vz:.2f})")

                # Force the exact velocity using enable_constant_velocity for one frame
                actor.enable_constant_velocity(carla.Vector3D(vx, vy, vz))
                actor.set_target_angular_velocity(carla.Vector3D(wx, wy, wz))

    # Let constant velocity apply for one tick
    world.tick()

    # Disable constant velocity and apply controls
    for rec in vehicles_state:
        vid = int(rec["id"])
        if vid not in present:
            continue

        actor = world.get_actor(vid)
        if actor:
            # Disable constant velocity
            actor.disable_constant_velocity()

            # Apply controls
            if apply_controls:
                ctrl = carla.VehicleControl(
                    throttle=float(rec.get("throttle", 0.0)),
                    steer=float(rec.get("steer", 0.0)),
                    brake=float(rec.get("brake", 0.0)),
                    hand_brake=bool(rec.get("hand_brake", False)),
                    reverse=bool(rec.get("reverse", False)),
                    gear=int(rec.get("gear", 0)),
                )
                actor.apply_control(ctrl)

    # Final tick
    world.tick()

    # Finally, reinit those two criteria only (prevents immediate FAILURE)
    reinit_inroute_and_blocked(manager.scenario_tree)

    if changed and not keep_sync:
        world.apply_settings(prev)

    start_manager_builder_only(manager)

### restore snapshot
pause_restore_resume(leaderboard_evaluator.client, leaderboard_evaluator.world, leaderboard_evaluator.manager, vehicles_state, mode="snapshot_strict", apply_controls=True, keep_sync=True)


Ego before impulse: cur=(-0.01, -0.00, -0.00), target=(12.25, 0.05, 0.01)


In [20]:
### third 50s 
# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

run_step_n = 50  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250903_145652

=== Running 50 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:53.130 -- System time = 27.856 -- Game time = 5.250 -- Ratio = 0.188x
Frame 101/10 - Test_0

Step 0:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:53.332 -- System time = 28.059 -- Game time = 5.300 -- Ratio = 0.189x
Frame 102/10 - Test_0

Step 1:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:53.538 -- System time = 28.265 -- Game time = 5.350 -- Ratio = 0.189x
Frame 103/10 - Test_0

Step 2:
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-09-03 14:56:53.720 -- System time = 28.447 -- Game time = 5.400 -- Ratio = 0.190x
Frame 104/10 - Test_0


In [21]:
for v in leaderboard_evaluator.manager.scenario_tree.iterate():
    print("name:", v.name, "status:",v.status)


name: LightsBehavior status: Status.RUNNING
name: RouteWeatherBehavior status: Status.RUNNING
name: ScenarioTriggerer status: Status.RUNNING
name: BackgroundActivity status: Status.RUNNING
name: WaitForBlackboardVariable: ScenarioRouteNumber0 status: Status.SUCCESS
name: LeaveSpaceInFront status: Status.SUCCESS
name: ChangeRoadBehavior status: Status.SUCCESS
name: ScenarioTimeout status: Status.RUNNING
name: WaitUntilInFront status: Status.RUNNING
name: DriveDistance status: Status.INVALID
name: End Condition status: Status.RUNNING
name: BasicAgentBehavior status: Status.RUNNING
name: Braking status: Status.INVALID
name: WaitForever status: Status.INVALID
name: Bicycle behavior status: Status.RUNNING
name: BasicAgentBehavior status: Status.RUNNING
name: Braking status: Status.INVALID
name: WaitForever status: Status.INVALID
name: Bicycle behavior status: Status.RUNNING
name: TriggerDistanceToVehicle status: Status.SUCCESS
name: Idle status: Status.SUCCESS
name: SetMaxSpeed status: Stat

In [22]:

_existing_vehicle_ids(leaderboard_evaluator.world)

{3695,
 3696,
 3697,
 3762,
 3763,
 3764,
 3765,
 3766,
 3767,
 3768,
 3769,
 3770,
 3771,
 3833,
 3834,
 3835,
 3836}

In [23]:
_existing_vehicle_ids(leaderboard_evaluator.manager.scenario.world)

{3695,
 3696,
 3697,
 3762,
 3763,
 3764,
 3765,
 3766,
 3767,
 3768,
 3769,
 3770,
 3771,
 3833,
 3834,
 3835,
 3836}